In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
import importlib

import itertools as it
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import statsmodels.api as sm

from itertools import product
from pandas_profiling import ProfileReport

from criteriaetl.utils.expansion_func import (get_value_counts_with_expansion_factor,
    get_percentage_table_with_expansion_factor)
from criteriaetl.utils.display_func import cdisplay, percentage_count_plot
from criteriaetl.utils.common_func import (get_weighted_complete_randomization_series_on_subset, 
    proportional_cut, weighted_qcut, get_partition_bool_columns_dict)
from criteriaetl.transformers.columns_base import (NameTransformer, 
    ReplaceTransformer, SelectTransformer, AssignTransformer)
from criteriaetl.transformers.rows_base import AggregateTransformer
from criteriaetl.transformers.fusion_base import MergeTransformer

from projectetl.utils.dataload import (load_survey_data_do, save_survey_with_pickle,
                                       load_survey_from_pickle)
from projectetl.utils.config import ENHOGAR_DIR

In [ ]:
survey_elected_raw = load_survey_data_do(ENHOGAR_DIR / 'Elegidos_ENHOGAR2016_PUB.sav',
                                         load_func=lambda path: pd.read_spss(str(path)))
survey_member_raw = load_survey_data_do(ENHOGAR_DIR / 'Personas_ENHOGAR2016_PUB.sav',
                                        load_func=lambda path: pd.read_spss(str(path)))
survey_house_raw = load_survey_data_do(ENHOGAR_DIR / 'Viviendas_ENHOGAR2016_PUB.sav',
                                       load_func=lambda path: pd.read_spss(str(path)))

In [ ]:
# hhogar 
# ---
# h402 = sexo
# h403 = edad
# h405 = parentesco
# h407 = estado civil (h407a = números)
# ----
# h409 = aprendió a leer y escribir
# h410 = donde aprendió a leer y escribir
# h411 = asiste o asistió a centro educativo
# h412 = nivel educativo más alto
# h413 = curso o año más alto que cursó
# h414 = asiste en el actual año escolar a algún centro educativo
# h415 = nivel educativo al que está asistiendo
# h416 = curso o año al que está asistiendo
# ---
# h501 = ¿Trabajó o realizó (nombre) alguna actividad económica por lo menos una hora la semana pasada?
# h502 = Aunque no trabajó la semana pasada ¿Tenía algún empleo, negocio o actividad?
# h503 = ¿La semana pasada (nombre) cultivó, cosechó o cuidó ganado u otros animales?
# h504 = ¿La semana pasada, elaboró (nombre) algún producto como artesanía o comida para vender?
# h505 = ¿La semana pasada, ayudó (nombre) a algún familiar en su negocio, empresa o finca?
# h506 = ¿La semana pasada (nombre) cosió, planchó, limpió casas, lavó ropa ajena o realizó otra actividad por paga?
# h507 = Durante las últimas cuatro semanas ¿Ha buscado (nombre) trabajo o estuvo tratando de establecer su propio negocio, actividad económica o empresa?
# h508 = ¿Por qué (nombre) no ha buscado trabajo?
# lh509 = ¿Podría (nombre) haber aceptado un trabajo la semana pasada si le hubieran ofrecido uno?
# lh510 = ¿Dispone (nombre) del tiempo y las condiciones necesarias para salir a trabajar?
# lh511 = La semana pasada, ¿Habría tenido (nombre) el tiempo y las condiciones necesarias para salir a trabajar?
# lh512 = ¿Ha trabajado (nombre) antes por paga o ganancia?
# lh513 = ¿Cuál fue la principal ocupación u oficio línea que desempeñó (nombre) durante la semana pasada o en el último trabajo que tuvo?
# lh514 = ¿A qué se dedica la empresa, negocio o institución en la que trabaja (nombre) actualmente o trabajó por última vez?
# lh515 = ¿En ese trabajo (nombre) es o era...
# lh516 = Regularmente, ¿Cuántas horas a la semana trabaja o trabajó (nombre) en su ocupación principal?
# h601 = ¿Tiene (nombre) PREG. algún seguro de salud, seguro médico o plan de salud?
# region
member_columns = [
    # Identifiers
    'hhogar', 'hzona', 'hprovi', 'h402', 'h403', 'h405', 'h407',
    # Education
    'h409', 'h410', 'h411', 'h412', 'h413', 'h414',
    'h415', 'h416',
    # Economic characteristics
    'h501', 'h502', 'h503', 'h504', 'h505', 'h506', 
    'h507', 'h508', 'lh509', 'lh510', 'lh511', 'lh512', 'lh513',
    'lh514', 'lh515', 'lh516',
    # Assurance
    'h601',
    # Stratos
    'hestrat', 'region', 'grupsec', 'zona', 'hmiembro',
    'hprovin', 'estratos',
    # Factors
    'fexpansion', 'fponderacion'
]

# h702a = Número de personas estándar mayores de 10 en el hogar
# h704 = edad
# h1001 = ¿Trabajó usted la semana pasada por paga o ganacia?
# h1002 = ¿Cuántos días trabajó usted durante la semana pasada?
# h1003 = ¿Cuántas horas trabajó usted diariamente la semana pasada?
# h1005 = ¿Estudia usted actualmente?
# h1009 = ¿Realiza usted algún tipo de estudio vocacional, técnico o profesional como música, pintura, computación u otra actividad similar?
# h1015 = Durante la semana pasada ¿Dedicó usted algún tiempo al cuidado de enfermos, ancianos, niños u otras personas, ya sean de su hogar u otro hogar?
# h1016 = Durante la semana pasada ¿Cuántos días le dedico al cuidado de esa o esas personas?
# h1017 = Durante la semana pasada ¿Cuántas horas le dedicó usted al cuidado de esa o esas personas diariamente o el último día que cuidó a esa o esas personas?
# h1030 = En la semana pasada, ¿Cuántos días dedicó usted tiempo para criar o cuidar animales o realizar algún cultivo o cosecha de productos para el consumo del hogar?
# h1032 = Durante la semana pasada, ¿Cuántos días dedicó usted a realizar algún trabajo sin paga en un negocio o establecimiento de comercio o producción?
# h1034 = En la semana pasada, ¿Cuántos días dedicó usted tiempo para realizar actividades domésticas en su hogar, como limpiar la casa, cocinar, lavar, fregar u otras labores domésticas?
# h1035 = En la semana pasada, ¿Cuántas horas dedicó usted a realizar actividades diariamente o el último día que las realizó la semana pasada?
# h1036 = En la semana pasada, ¿Cuántos días dedicó usted tiempo para realizar actividades domésticas sin paga en otro hogar, como limpiar la casa, cocinar, lavar, fregar u otras labores domésticas?
# h1037 = Durante la semana pasada, ¿Cuántas horas dedicó usted a realizar esas actividades diariamente o el último día que las ralizó la semana pasada?
elected_columns = [
    'h703', 'h702a',
    'h704', 'h1001', 'h1002', 'h1003', 'h1005',
    'h1009', 'h1030', 'h1032',
    'fexpansion_tic', 'fponderacion_tic',
    # Target
    'h1015', 'h1016', 'h1017', 'h1034', 'h1035',
    'h1036', 'h1037'
]

# Processing surveys

In [ ]:
basic_cols = [
    'hhupm', 'hhviv', 'hhogar'
]

## Assign Transformer

In [ ]:
assign_member_map = {
    'temp': lambda _: 1,
    'h703': lambda df, cols=basic_cols: df.groupby(cols)['temp'].transform('cumsum')
}

assign_member_transformer = AssignTransformer(assign_member_map)
survey_member_assigned = assign_member_transformer.transform(survey_member_raw)

In [ ]:
survey_member_assigned.head()

## Merge Transformer

In [ ]:
merge_member_elected_transformer = MergeTransformer(lambda : survey_member_assigned,
                                                    merge_kwargs={
                                                        'on': basic_cols + ['h703'],
                                                        'suffixes': ('', '_other')
                                                    })
survey_elected_merged = merge_member_elected_transformer.transform(survey_elected_raw)

In [ ]:
survey_elected_merged

In [ ]:
survey_elected_merged['']

## Name Transformer

In [ ]:
name_map = {
    'hhogar': ''
    'h402': 'sexo'
    'h403': 'edad'
    'h405': 'parentesco'
    'h407': 'estado_civil' (h407a = números)
    'h409': aprendió a leer y escribir
    'h410': donde aprendió a leer y escribir
    'h411': asiste o asistió a centro educativo
    'h412': nivel educativo más alto
    'h413': curso o año más alto que cursó
    'h414': asiste en el actual año escolar a algún centro educativo
    'h415': nivel educativo al que está asistiendo
    'h416': curso o año al que está asistiendo
    'h501': ¿Trabajó o realizó (nombre) alguna actividad económica por lo menos una hora la semana pasada?
    'h502': Aunque no trabajó la semana pasada ¿Tenía algún empleo, negocio o actividad?
    'h503': ¿La semana pasada (nombre) cultivó, cosechó o cuidó ganado u otros animales?
    'h504': ¿La semana pasada, elaboró (nombre) algún producto como artesanía o comida para vender?
    'h505': ¿La semana pasada, ayudó (nombre) a algún familiar en su negocio, empresa o finca?
    'h506': ¿La semana pasada (nombre) cosió, planchó, limpió casas, lavó ropa ajena o realizó otra actividad por paga?
    'h507': Durante las últimas cuatro semanas ¿Ha buscado (nombre) trabajo o estuvo tratando de establecer su propio negocio, actividad económica o empresa?
    'h508': ¿Por qué (nombre) no ha buscado trabajo?
    'lh509': ¿Podría (nombre) haber aceptado un trabajo la semana pasada si le hubieran ofrecido uno?
    'lh510': ¿Dispone (nombre) del tiempo y las condiciones necesarias para salir a trabajar?
    'lh511': La semana pasada, ¿Habría tenido (nombre) el tiempo y las condiciones necesarias para salir a trabajar?
    'lh512': ¿Ha trabajado (nombre) antes por paga o ganancia?
    'lh513': ¿Cuál fue la principal ocupación u oficio línea que desempeñó (nombre) durante la semana pasada o en el último trabajo que tuvo?
    'lh514': ¿A qué se dedica la empresa, negocio o institución en la que trabaja (nombre) actualmente o trabajó por última vez?
    'lh515': ¿En ese trabajo (nombre) es o era...
    'lh516': Regularmente, ¿Cuántas horas a la semana trabaja o trabajó (nombre) en su ocupación principal?
    'h601':¿Tiene (nombre) PREG. algún seguro de salud, seguro médico o plan de salud?
    region
}

## Final dataframe

In [ ]:
survey_elected_final = survey_elected_merged[member_columns + elected_columns]
cdisplay(survey_elected_final)

In [ ]:
len(survey_elected_final.columns.unique())

# EDA

## Pandas profiling

In [ ]:
report = ProfileReport(survey_elected_final, config_file='config.yml')

In [ ]:
report.to_file('report.html')

## Pearson correlation

In [ ]:
survey_elected_final['h1015'].count()

In [ ]:
correlation = survey_elected_final.corr(method='spearman')
sns.heatmap(correlation, xticklabels=correlation.columns, yticklabels=correlation.columns)